# E8 - 프로젝트 - Movielens 영화 추천 실습

## 실습환경 구성 : 작업 디렉토리 생성, 데이터셋 다운로드

* 이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.  
* 이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.  
 - 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.  
 - 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.  
 - 별점을 시청횟수로 해석해서 생각하겠습니다.  
 - 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.  
  
* MovieLens 1M Dataset 데이터셋의 경우는 크기가 작아서 아래와 같이 직접 다운 받고 압축해제 하겠습니다.  
1) wget으로 데이터 다운로드  
wget http://files.grouplens.org/datasets/movielens/ml-1m.zip  
2) 다운받은 데이터를 작업디렉토리로 옮김  
mv ml-1m.zip ~/aiffel/recommendata_iu/data  
3) 작업디렉토리로 이동  
cd ~/aiffel/recommendata_iu/data  
4) 압축 해제  
unzip ml-1m.zip

## 데이터 준비와 전처리

* Movielens 데이터는 ratings.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [1]:
import pandas as pd
import os

# ratings 파일이 우리가 다룰 데이터 : 파일 경로 저장하고, ratings 목록의 각 행은 '사용자아이디/영화아이디/평점/타임스탬프'
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']

# ratings파일(데이터셋) 읽어오기
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 데이터셋에서 평점 3점 이상인 데이터만 남기기
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

# 원본 데이터 수, 평점 3점 이상인 데이터 수
print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
# 걸러진 후 남은 데이터 비율
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꾸기
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
# 카운트 확인
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 사용하는 컬럼만 남기고 `timestamp` 항목 제거

using_cols = ['user_id', 'movie_id', 'count']
ratings = ratings[using_cols]
ratings.head(10)

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어오기
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python',encoding='latin1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


#검색을 쉽게하기 위해 영화제목(title) 문자열을 소문자로 바꿔주기  
movies['title'] = movies['title'].str.lower()  
  
#검색을 쉽게하기 위해 장르(genre) 문자열을 소문자로 바꿔주기  
movies['genre'] = movies['genre'].str.lower()  
  
movies.head(10)

## 데이터 분석

* ratings에 있는 유니크한 영화 개수  
* ratings에 있는 유니크한 사용자 수  
* 가장 인기 있는 영화 30개(인기순)

In [7]:
# 영화 개수
ratings['movie_id'].nunique()

3628

In [8]:
# 사용자 수
ratings['user_id'].nunique()

6039

In [9]:
# 인기 있는 영화 30개  
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [10]:
# 유저별 몇 편의 영화를 보는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [11]:
# 유저별 영화평점 중앙값에 대한 통계
user_median = ratings.groupby('user_id')['count'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: count, dtype: float64

## 내가 선호하는 영화 5가지 골라서 rating에 추가

* 우리는 본인의 음악 취향과 가장 유사한 아티스트를 추천받고 싶습니다. 유튜브 뮤직 등 추천시스템들은 이를 위해서 처음 가입하는 사용자의 취향과 유사한 아티스트 정보를 5개 이상 입력받는 과정을 거치게 하는 경우가 많습니다  
* 우리도 동일한 과정이 필요합니다. 우리가 만든 추천시스템의 추후 검증 과정을 위해, '나는 A를 좋아한다.'라는 정보를 위 데이터셋에 5개 이상 추가해 주어야 합니다. 저는 아래와 같이 제가 좋아하는 유명한 아티스트들을 기존의 데이터에 추가해서 넣겠습니다.

In [12]:
# 본인이 좋아하시는 영화 데이터 추가 (단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞추기)
my_favorite = ['Toy Story (1995)', 'Sense and Sensibility (1995)', 'While You Were Sleeping (1995)',
               'Saving Grace (2000)', 'Star Wars: Episode I - The Phantom Menace (1999)']
# 메타데이터를 통한 영화 제목별 아이디 확인
my_fav_id = [1, 17, 339, 3831, 2628]

# 10이라는 user_id가 위 영화를 평점 5 줬다고 가정
my_movielist = pd.DataFrame({'user_id': ['10']*5, 'movie_id':my_fav_id,
                             #'title':my_favorite,
                             'count':[5]*5})

if not ratings.isin({'user_id':['10']})['user_id'].any():  # user_id에 '10'이라는 데이터가 없다면
    ratings = ratings.append(my_movielist)                  # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 
ratings.tail(10)     # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,10,1,5
1,10,17,5
2,10,339,5
3,10,3831,5
4,10,2628,5


## 모델에 활용하기 위한 전처리

* 사람이 태어나면 주민등록번호가, 학교에 가면 출석번호가 있듯이 데이터의 관리를 쉽게 하기 위해 번호를 붙여주고 싶습니다. 우리가 다루는 데이터에서는 user와 movie 각각에 번호를 붙이고 싶습니다. 보통 이런 작업을 indexing이라고 합니다. 추천시스템, 자연어처리에서 자주 하는 작업들이라 직접 해보시면 좋겠습니다.  
  
* (참고) pandas.DataFrame.unique()은 특정 컬럼에 포함된 유니크한 데이터만 모아 줍니다. indexing 작업을 위해 매우 유용합니다.

In [13]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [14]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['10'])    # 6040명의 유저 중 마지막으로 추가된 유저, 6039가 나와야 함  
print(movie_to_idx[339])

6039
579


In [15]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie_id 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

ratings

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,count
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
0,6039,40,5
1,6039,357,5
2,6039,579,5
3,6039,1465,5


* data의 user_id와 movie_id 컬럼 내 값들이 모두 정수 인덱스 값으로 잘 변경되었나요? 이것으로 훈련을 위한 전처리가 완료!!

## 사용자의 명시적/암묵적 평가

In [16]:
# 평점 3이하인 데이터의 비율을 보는 코드
only_one = ratings[ratings['count']<4]
one, all_ratings = len(only_one), len(ratings)
print(f'{one},{all_ratings}')
print(f'Ratio of only_one over all data is {one/all_ratings:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

261197,836483
Ratio of only_one over all data is 31.23%


* 우리가 앞으로 만들어갈 모델에서는 암묵적 데이터의 해석을 위해 다음과 같은 규칙을 적용할 것입니다.  
  
 - 평점 4 이상이면 선호한다고 판단한다.  
 - 평점 높은 영화에 대해 가중치를 주어서 더 확실히 좋아한다고 판단한다.

## CSR(Compressed Sparse Row) Matrix

* 이제 우리의 data를 CSR Matrix에 맞게 바꿔보겠습니다.  
* DataFrame을 만드는 방식이 다양하듯 csr_matrix를 만드는 방법은 다양하게 있습니다.  
* 현재 data와 같은 데이터 구조에 적합한 방식은 위 링크에서 4번째로 설명하고 있는 방법입니다.  
  
* https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html  
* 링크의 네 번째 방법 이용할 것임

In [17]:
print(ratings['count'])

0    5
1    3
2    3
3    4
4    5
    ..
0    5
1    5
2    5
3    5
4    5
Name: count, Length: 836483, dtype: int64


In [18]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련

* 이전 스텝에서 설명한 Matrix Factorization 모델을 implicit 패키지를 사용하여 학습해 봅시다.   
  
 - implicit 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지입니다.  
 - 이 패키지에 구현된 als(AlternatingLeastSquares) 모델을 사용하겠습니다. Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 것으로 알려져 있습니다.  
 - https://github.com/benfred/implicit

In [19]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

* AlternatingLeastSquares 클래스의 __init__ 파라미터를 살펴보겠습니다. 1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 3. use_gpu : GPU를 사용할 것인지 4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지  
  
* 1,4를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 합니다.

In [20]:
# Implicit AlternatingLeastSquares 모델의 선언

als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [21]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [22]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [23]:
# 모델이 1)나의 벡터와 영화 saving grace 벡터를 어떻게 만들고 있는지와 2)두 벡터를 곱하면 어떤 값이 나오는지 살펴보기
# 339 = 'Saving Grace (2000)'
ten, grace = user_to_idx['10'], movie_to_idx[339]
ten_vector, grace_vector = als_model.user_factors[10], als_model.item_factors[339]

print('슝=3')

슝=3


In [24]:
ten_vector

array([ 1.9920932 ,  3.9760826 , -1.2460871 ,  2.8629994 ,  1.3867748 ,
       -0.4271622 , -5.047673  ,  0.7253164 ,  1.5692291 ,  3.9563916 ,
       -0.05733378, -4.0875    ,  2.463487  ,  2.0214462 , -1.849436  ,
        1.5113976 , -0.94835806,  1.4959528 , -1.04071   ,  2.1403813 ,
       -1.1373117 ,  0.7231477 , -1.9729098 , -3.0732043 ,  0.98216987,
        3.0173025 , -1.9655831 , -1.2058647 , -1.1691239 , -0.6857061 ,
        0.5497724 ,  2.0189073 , -1.0728357 ,  0.06284408, -1.4213376 ,
        0.7270098 ,  0.3076852 ,  0.07959504,  0.5105121 , -1.0762588 ,
        0.6297363 ,  0.0827704 , -0.4580571 ,  0.7595295 , -0.5119732 ,
       -1.510907  , -0.46914563,  0.06610866, -0.16989468,  0.8311259 ,
        2.1517398 , -2.1741517 ,  1.524773  ,  1.4530375 , -0.27634045,
        2.6481056 ,  2.9055145 ,  0.674109  , -0.6367054 ,  0.17417815,
       -0.32602614, -0.7020556 , -0.63514435, -0.22173958, -0.51456136,
        0.95673054,  0.29039183, -0.6603044 ,  2.5095441 , -1.79

In [25]:
grace_vector

array([ 0.00311455,  0.00934946,  0.01705586, -0.00420465,  0.01967909,
       -0.02044237,  0.00869611,  0.00822105,  0.0032412 ,  0.00193341,
        0.02744444,  0.00098494,  0.03023586,  0.00791752,  0.0146353 ,
        0.00086517, -0.00759674,  0.00234847,  0.00843734,  0.02079288,
       -0.00303815,  0.00515965,  0.01222022,  0.00728688,  0.01759453,
       -0.00612016,  0.00925481,  0.00503336, -0.00051784, -0.00491138,
        0.02211715, -0.0024479 ,  0.00669168,  0.0226726 ,  0.00372811,
       -0.01181681,  0.00973588,  0.0205172 ,  0.00533723,  0.00555046,
        0.02437565,  0.00388896,  0.00184339,  0.00289176,  0.00619891,
        0.03414315,  0.01534401, -0.0172581 ,  0.01204433,  0.00862856,
        0.00071608,  0.01205433, -0.00088676,  0.01314469,  0.01359866,
        0.00713862, -0.00794906, -0.00126522, -0.01728743,  0.01631886,
       -0.0054023 ,  0.01524019,  0.01037241,  0.00673678,  0.0319476 ,
       -0.01136479,  0.02331803,  0.01921112,  0.01186833, -0.02

In [26]:
# ten과 grace를 내적하는 코드
np.dot(ten_vector, grace_vector)

0.07908103

In [27]:
# 모델이 나의 '356::Forrest Gump (1994)::Comedy|Romance|War'에 대한 선호도를 어떻게 예측할지 한 번 보겠습니다

forrest = movie_to_idx[356]
forrest_vector = als_model.item_factors[356]
np.dot(ten_vector, forrest_vector)

0.39557433

## 비슷한 아티스트 찾기 + 유저에게 추천하기

### 비슷한 아티스트 찾기

In [28]:
# AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 선호하는 영화를 찾습니다. 
# 처음으로는 제가 좋아하는 '1::Toy Story (1995)::Animation|Children's|Comedy'로 찾아보겠습니다.

favorite_movie = 'Toy Story (1995)'
fav_movie_id = 1
movie_id = movie_to_idx[fav_movie_id]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

# (movie id, 유사도) Tuple 로 반환하고 있습니다.

[(40, 1.0000002),
 (50, 0.79627335),
 (4, 0.61659837),
 (322, 0.5721992),
 (33, 0.5652337),
 (110, 0.50152683),
 (330, 0.48096478),
 (10, 0.4400014),
 (255, 0.39560765),
 (34, 0.388531),
 (20, 0.37978664),
 (16, 0.36072043),
 (191, 0.35246134),
 (32, 0.3505131),
 (126, 0.34729245)]

In [29]:
# movie idx를 다시 movie_id로 매핑 시키기
# movie_to_idx 를 뒤집어, index로부터 movie_id를 얻는 dict를 생성합니다. 

idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

[1,
 3114,
 2355,
 34,
 588,
 1265,
 364,
 595,
 1923,
 1907,
 2321,
 2687,
 2081,
 1566,
 2396]

#movies.dat 파일에서 영화아이디로 제목 불러오고 싶은데 실패!! 뭔소린지 모르겠다... ㅠㅠ 일단 넘어가기...  
sim_movie_id = [idx_to_movie[i[0]] for i in similar_movie]  
sim_movie_title = [movies[i[0]] for i in sim_movie_id]

In [30]:
# 몇 번 더 반복해서 확인하기 위해 위의 코드를 함수로 만들고 확인해보겠습니다.

def get_similar_movie(movie_id: int):
    movie_idx = movie_to_idx[movie_id]
    similar_movie = als_model.similar_items(movie_idx)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [31]:
# '1580::Men in Black (1997)::Action|Adventure|Comedy|Sci-Fi'으로 확인

get_similar_movie(1580)

[1580, 480, 589, 2916, 780, 2571, 1527, 989, 1573, 1544]

In [32]:
# '1409::Michael (1996)::Comedy|Romance' 확인

get_similar_movie(1409)

[1409, 2266, 237, 276, 1353, 236, 339, 1835, 252, 830]

### 유저에게 아티스트 추천하기

* AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 내가 좋아할 만한 영화를 추천받기  
* filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.

In [33]:
user = user_to_idx['10']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(50, 0.40557933),
 (322, 0.33148998),
 (64, 0.29023215),
 (4, 0.26890722),
 (352, 0.2678976),
 (44, 0.26383168),
 (126, 0.25885743),
 (474, 0.23306894),
 (626, 0.2283763),
 (117, 0.2211415),
 (336, 0.20866305),
 (666, 0.19522803),
 (450, 0.18794954),
 (27, 0.17772284),
 (670, 0.17648135),
 (1212, 0.1705055),
 (1200, 0.16943939),
 (38, 0.16648905),
 (340, 0.1661968),
 (33, 0.16543427)]

In [34]:
[idx_to_movie[i[0]] for i in movie_recommended]

# '3114::Toy Story 2 (1999)::Animation|Children's|Comedy' 추천
# '1210::Star Wars: Episode VI - Return of the Jedi (1983)::Action|Adventure|Romance|Sci-Fi|War' 추천
# '2396::Shakespeare in Love (1998)::Comedy|Romance' 추천
# '260::Star Wars: Episode IV - A New Hope (1977)::Action|Adventure|Fantasy|Sci-Fi' 추천
# '2424::You've Got Mail (1998)::Comedy|Romance' 추천

[3114,
 34,
 1210,
 2355,
 838,
 260,
 2396,
 3751,
 2424,
 1196,
 597,
 539,
 2324,
 1721,
 3175,
 852,
 11,
 2762,
 1569,
 588]

In [35]:
# 모델은 왜 '토이스토리2'를 추천해줬을까? 
# AlternatingLeastSquares 클래스에 구현된 explain 메소드를 사용하면 내가 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인 가능

toystory2 = movie_to_idx[3114]
explain = als_model.explain(user, csr_data, itemid=3114)

# 이 method는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 됩니다.)를 반환

In [36]:
# 어떤 영화들이 이 추천에 얼마나 기여하고 있는 걸까요?

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]


# my_favorite = ['Toy Story (1995)', 'Sense and Sensibility (1995)', 'While You Were Sleeping (1995)',
#               'Saving Grace (2000)', 'Star Wars: Episode I - The Phantom Menace (1999)']
# my_fav_id = [1, 17, 339, 3831, 2628]

# 'sense and sensibility'와 'toy story'가 가장 크게 기여 

[(17, 0.004053547849301979),
 (1, 0.0010742385180326035),
 (3831, -0.002228433487722224),
 (339, -0.0029556103327386474),
 (2628, -0.005732298482521868)]

## 회고

매번 코드 문제에서 문제가 발생한다. 코딩에 익숙하면 영화아이디로 영화제목 읽어오는 방법으로 쉽게 추천받은 영화들을 확인할 수 있을 텐데 이 부분을 해결하지 못한 점이 너무 아쉽다. 이후에 찾아보고 공부하는 걸로...